# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка.\
Входные данные от банка — статистика о платёжеспособности клиентов.\
Нужно разобраться, какие факторы влияют на факт погашения кредита в срок.\
Есть гипотеза, что семейное положение и количество детей клиента влияют на факт погашения кредита в срок.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:

children — количество детей в семье\
days_employed — общий трудовой стаж в днях\
dob_years — возраст клиента в годах\
education — уровень образования клиента\
education_id — идентификатор уровня образования\
family_status — семейное положение\
family_status_id — идентификатор семейного положения\
gender — пол клиента\
income_type — тип занятости\
debt — имел ли задолженность по возврату кредитов\
total_income — ежемесячный доход\
purpose — цель получения кредита

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/internet.csv')
data.to_csv('internet.csv', index=False)

data = pd.read_csv('/datasets/messages.csv')
data.to_csv('messages.csv', index=False)

data = pd.read_csv('/datasets/users.csv')
data.to_csv('users.csv', index=False)

data = pd.read_csv('/datasets/tariffs.csv')
data.to_csv('tariffs.csv', index=False)

## Шаг 1. Общая информация

In [ ]:
#импорт всех необходимых библиотек

import pandas as pd

from pymystem3 import Mystem
m = Mystem()

from collections import Counter

In [ ]:
df = pd.read_csv('/datasets/data.csv')
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19092,0,NaN,53,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,получение образования
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
13098,0,NaN,41,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,свой автомобиль
7782,0,-2404.645709,59,высшее,0,гражданский брак,1,F,компаньон,0,372580.183620,на проведение свадьбы
21306,3,-260.311966,32,среднее,1,женат / замужем,0,F,сотрудник,0,64925.398860,покупка жилья


### общая информация

In [ ]:
df.info() #общая инфо о базе

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


в базе 12 столбцов, 21525 строк
2 столбца с нецелыми значениями: days_employed, total_income

In [ ]:
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15385,0,-545.122448,29,среднее,1,гражданский брак,1,M,сотрудник,0,303907.111529,на проведение свадьбы
1569,0,NaN,23,высшее,0,гражданский брак,1,F,компаньон,0,NaN,на проведение свадьбы
4317,0,-357.207058,48,среднее,1,вдовец / вдова,2,F,компаньон,0,63721.655306,покупка жилья для семьи
9842,0,-3311.544263,52,среднее,1,Не женат / не замужем,4,F,компаньон,0,220726.293490,недвижимость
1347,2,-5193.652233,37,начальное,3,женат / замужем,0,M,сотрудник,0,94624.091076,строительство жилой недвижимости


In [ ]:
df.isna().sum()#пропуски есть в 2 столбцах

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
df.duplicated().sum()# 54 строки - дубликаты

54

### знакомство с данными по столбцам

In [ ]:
#стаж в годах - мин, максимум, среднее и медиана - есть отрицательные и аномально большие значения, и нужно привести к int
print('min (лет): ',int(df['days_employed'].min()/365),'\n','max (лет): ', int(df['days_employed'].max()/365),'\n','mean (лет): ', int(df['days_employed'].mean()/365),'\n','median (лет): ', int(df['days_employed'].median()/365))

min (лет):  -50 
 max (лет):  1100 
 mean (лет):  172 
 median (лет):  -3


In [ ]:
# убираем отрицательные значения в стаже и снова проверка
df['days_employed'] = df['days_employed'].abs()
df['days_employed'].describe().astype('int')
#print(int(df['days_employed'].min()/365), int(df['days_employed'].max()/365), int(df['days_employed'].mean()/365), int(df['days_employed'].median()/365))

count     19351
mean      66914
std      139030
min          24
25%         927
50%        2194
75%        5537
max      401755
Name: days_employed, dtype: int64

In [ ]:
#возраст - мин, максимум, среднее и медиана - есть нулевой возраст, средний возраст 43 года, верхняя граница - 75 лет
df['dob_years'].describe().astype('int')
#print(df['dob_years'].min(), df['dob_years'].max(), int(df['dob_years'].mean()), int(df['dob_years'].median()))

count    21525
mean        43
std         12
min          0
25%         33
50%         42
75%         53
max         75
Name: dob_years, dtype: int64

In [ ]:
df[df['dob_years']==0]['dob_years'].count() #101 строка с нулевым возрастом

101

In [ ]:
#количество детей - варианты значений и их количество - есть отрицательные значения(47 строк), и аномалия 20
df.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [ ]:
# замена значений -1 и 20 на 1 и 2 соответственно - в основном клиенты без детей
df['children'] = df['children'].replace(-1,1) 
df['children'] = df['children'].replace(20,2)
df.groupby('children')['children'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [ ]:
# образование - приведем все значения к нижнему регистру
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [ ]:
df['education'] = df['education'].str.lower() # приведем все значения к нижнему регистру - среднее образование у большинства клиентов
df.groupby('education')['education'].count()

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

In [ ]:
#семейное положение - варианты значений и их количество - большинство клиентов в браке
df.groupby('family_status')['family_status'].count()

family_status
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
Name: family_status, dtype: int64

In [ ]:
#пол - варианты значений и их количество - женщин больше в 2 раза
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [ ]:
df['gender'] = df['gender'].replace('XNA','F') # замена значения XNA на F, как наиболее распространенный в базе.
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [ ]:
df.duplicated().sum()# строки - дубликаты - количество дубликатов увеличилось

71

In [ ]:
#источник дохода - варианты значений и их количество - работа по найму у большинства 
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [ ]:
df['purpose'].unique() # цель кредита - очень много вариантов ответа

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
#наличие задолженности - варианты значений и их количество - задолженность у 8% клиентов
print(df['debt'].value_counts())
100*1741/21527

0    19784
1     1741
Name: debt, dtype: int64


8.087518000650347

In [ ]:
# total=21525
# display(14149/total) #клиентов без детей (14149 из 21525 строк), 
# display(15233/total) # со средним образованием (15233 строк), 
# display((12380+4177)/total)  #состоят в браке (12380+4177 гражданский брак),
# display(14236/total) #женщин в 2 раза больше, чем мужчин (14236 / 7288),
# display((11119+1459)/total)  #работают по найму (11119+1459 госслужащих), компаньонов= 5085, достаточно много пенсионеров = 3856,14149 из 21525
# 3856/total

In [ ]:
print(int(df.query('children == 0').shape[0] / df.shape[0]*100), '% клиентов без детей')

print(int(df[df['education'].str.match('среднее')].shape[0] / df.shape[0]*100), '% клиентов cо средним образованием')
print(int(df[df['income_type'].str.match('сотрудник')].shape[0] / df.shape[0]*100), '% клиентов работают по найму')
print(int(df[df['gender'].str.match('F')].shape[0] / df.shape[0]*100), '% клиентов - женщины')

print(int(df[(df['family_status'] == 'женат / замужем') | (df['family_status'] == 'гражданский брак')].shape[0] / df.shape[0]*100), '% клиентов состоят в браке')


65 % клиентов без детей
70 % клиентов cо средним образованием
51 % клиентов работают по найму
66 % клиентов - женщины
76 % клиентов состоят в браке


**Вывод**

в базе 12 столбцов, 21524 записи(строки)\
54 строки-дубликата, после минимальной коррекции данных дубликатов стало больше = 71 строка.\
2 столбца с пропусками: days_employed, total_income.2174 строки, это 10%\
\
Характеристики клиентов банка:
- женщин в 2 раза больше, чем мужчин (14236 / 7288),
- 70% со средним образованием (свыше 15000 строк), 
- 77% cостоят в браке (12380+4177 гражданский брак),
- 66% клиентов без детей (14149 из 21525 строк), 
- 58% работают по найму (11119+1459 госслужащих), 24% =компаньоны (5085), достаточно много пенсионеров = 18% (3856),
- возраст заемщика от 19 до 75 лет, средний = 43,
- ежемесячный доход от 20тр до 2 млн рублей, среднее = 167тр, и только 2% (528 строк/клиентов) с доходом >500тр,
- цель кредита пока сложно охарактеризовать - нужно сгруппировать данные.
задолженность есть только у 8% клиентов (1741 от 21524).

- при первом знакомстве с данными сложилось впечатление, что семейное положение и количество детей не оказывают большого влияния на возникновение задолженности по кредиту.
- можно посмотреть данные детальнее по более мелким категориям.
- наличие пропусков в других столбцах не влияет на ответ по этому вопросу.

\
проведена коррекция данных:
- children - есть отрицательные значения(47 строк), и аномалия 20 (76 строк), которые были исправлены на 1 и 2 соответственно. 20 детей - возможный вариант, но 76 обращений в одном банке маловероятно (был бы больше разброс значений).
- education - привести к нижнему регистру,
- gender - аномалия XNA, которую можно бы и удалить, но проведена замена на F, как наиболее распространенный,
- days_employed - замена отрицательных значений.

нужна коррекция данных:
- dob_years - есть нулевой возраст
- days_employed - есть аномально большие значения и нужно привести к int
- purpose - очень много вариантов, нужно унифицировать
- total_income - нужно привести к int
- 2 столбца с пропусками: days_employed, total_income.2174 строки, это 10%
- 71 строка с дубликатами


## Шаг 2. Предобработка данных

### Обработка пропусков

- 1 столбец с пропуском возраста 'dob_years'=0\
 101 строка - это очень мало, можно даже не пересчитывать средний возраст, а просто заменить все нули в этом столбце на среднее значение


- 2 столбца с пропусками NaN: days_employed, total_income.2174 строки, это 10%\
причиной появления пропусков может быть реальное отсутствие стажа и, как следствие, дохода, \
но эту версию опровергают варианты занятости "сотрудник", "госслужащий" и "компаньон".\
значит, причина пропуска - ошибка заполнения.\
количество пропусков незначительное - всего 10%, можно удалить эти строки или заменить значения:
 - заменить доход на среднее по группе "тип занятости"
 - считать стаж как возраст минус 18 лет

#### коррекция данных по возрасту 'dob_years'

In [ ]:
# заменяем нулевой возраст на среднее значение
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].mean()
df['dob_years'].min() #проверка, что замена прошла успешно

19.0

<div class="alert alert-info">
если оставить 0, то средний возраст клиента будет занижен, а также это может сказаться на каких-нибудь других расчетах с использованием возраста (деление на ноль, категоризация по возрасту и пр)
    
но так как таких строк мало (< 1%), то их удаление или коррекция не сильно повлияют на общие выводы
</div>

#### коррекция данных по стажу 'days_employed' - аномально большие значения

In [ ]:
# аномальные значения в столбце "days_employed"
# поиск границы аномалии - стаж в годах не может быть больше max возраста заемщика 75лет минус 18лет. 
display((75-18)*365) # max возраст 75 лет = 20805 дней
display(df[df['days_employed'] >20805]['days_employed'].count()) #сколько таких строк - их 3445
display(df[df['days_employed'] >20805]['days_employed'].min()) #где граница - 328728 дней = 900 лет
int(328728/365) # 328728 дней = 900 лет

20805

3445

328728.72060451825

900

In [ ]:
#подбор способа коррекции данных
display(328728/100/365) # можно поделить на 100, если причина в опечатке и тогда 900лет станут 9 годами
display(328728/60/365) # можно поделить на 60, если эта величина была расчетная, и данные были представлены в минутах - правильный ответ 15 лет

9.006246575342466

15.01041095890411

In [ ]:
df['days_employed'].describe().astype('int') # проверка перед изменением

count     19351
mean      66914
std      139030
min          24
25%         927
50%        2194
75%        5537
max      401755
Name: days_employed, dtype: int64

In [ ]:
#функция по замене аномально больших значений 
def days_correct(value):
    if value >20805:
        value = value/100 # можно поменять этот коэффициент после выяснения причины аномалии
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(days_correct) # исправление аномального стажа
df['days_employed'].describe().astype('int') # проверка

count    19351
mean      2583
std       2149
min         24
25%        927
50%       2194
75%       3658
max      18388
Name: days_employed, dtype: int64

#### коррекция данных по стажу 'days_employed' - проверить разницу между возрастом и стажем > 18? 

In [ ]:
df1=df.dropna() # создаем копию базы без пропусков
df1['years_employed'] = df1['days_employed']/365 # стаж переведен в года
df1['diff_age_employed'] = df1['dob_years'] - df1['years_employed'].astype('int') # во сколько лет заемщик начал работать

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
display(df1['diff_age_employed'].sort_values().head()) #минимальный возраст начала стажа - 11-13 лет, такое бывает, данные не требуют коррекции.

5708     11.0
16335    11.0
397      12.0
13531    13.0
19902    13.0
Name: diff_age_employed, dtype: float64

In [ ]:
df1['diff_age_employed'].describe().astype('int') # проверка пройдена, основная масса клиентов приступила к работе после 27 лет.

count    19351
mean        36
std         11
min         11
25%         27
50%         36
75%         46
max         71
Name: diff_age_employed, dtype: int64

основная масса клиентов приступила к работе после 27 лет, что тоже немного странно, учитывая, что 2/3 клиентов это женщины со средним образованием (нет 5 лет ВУЗа) и без детей (без декрета). Почему возникла потребность в кредите после 10-20 лет без работы? Успешное замужество, т.к. сейчас они в браке, или на обеспечении родителей? Инфантильность клиента может повлиять на факт погашения кредита в срок.\
Эта информация может пригодиться просто для формирования подхода к клиентам, т.к. доля должников по базе достаточно низкая = 8%

#### коррекция данных по стажу 'days_employed' - замена пропусков в стаже на возраст минус 18 лет

In [ ]:
# замена пропусков в стаже на возраст минус 18 лет
df.loc[df['days_employed'].isna(), 'days_employed'] = (df['dob_years'] - 18)*365 # стаж указан в днях
df['days_employed'].isna().sum() # проверка

0

#### коррекция данных по доходу 'total_income' - заменить доход на среднее по типу занятости

In [ ]:
# пропуски есть во всех типах занятости
df[df['total_income'].isna()].groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

In [ ]:
#средний доход значительно отличается по разным типам занятости.
# расчет среднего, мин и макс по каждой группе. Сортировка по убыванию.
df_income_type = round(df.groupby('income_type').agg({'total_income': ['mean', 'min', 'max']})).sort_values(by = ('total_income','mean'), ascending = False)
df_income_type
#df.groupby('income_type')['total_income'].describe() # более короткий способ, но менее наглядный вывод на экран

total_income                     
                        mean       min        max
income_type                                      
предприниматель     499163.0  499163.0   499163.0
компаньон           202417.0   28703.0  2265604.0
госслужащий         170898.0   29200.0   910451.0
сотрудник           161380.0   21368.0  1726276.0
пенсионер           137127.0   20667.0   735103.0
безработный         131340.0   59957.0   202723.0
студент              98202.0   98202.0    98202.0
в декрете            53829.0   53829.0    53829.0

In [ ]:
# % должников в базе с пропусками такой же, как и в общей базе = 8%  это еще одна причина, почему можно было бы просто удалить эти строки, 
#но мы просто позволим себе рассчитывать среднее по общей базе, без удаления строк с пропусками
print(df[df['total_income'].isna()].groupby('debt')['debt'].count())
round(100*170/2174,2) 

debt
0    2004
1     170
Name: debt, dtype: int64


7.82

In [ ]:
# по каждому элементу из столбца "тип занятости" рассчитываем средний доход, а потом заменяем пропущенные значения на этот средний доход
for inc_type in df['income_type'].unique():
  replace_na_value = df[df['income_type'] == inc_type]['total_income'].mean()  # средний доход
  df.loc[df['total_income'].isna() | (df['income_type'] == inc_type), 'total_income'] = replace_na_value
  print(inc_type, int(replace_na_value))

сотрудник 161380
пенсионер 139725
компаньон 198317
госслужащий 169939
безработный 131339
предприниматель 330271
студент 98201
в декрете 53829


In [ ]:
#df["total_income"] = df.groupby("income_type").transform(lambda x: x.fillna(x.mean())) #проверка

In [ ]:
#df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('mean')) #проверка

In [ ]:
df.head() # проверка, что пропущенных значений больше нет

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,161380.260488,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,161380.260488,приобретение автомобиля
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,161380.260488,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,161380.260488,дополнительное образование
4,0,3402.660720,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,139725.080901,сыграть свадьбу


In [ ]:
df.isna().sum() # проверка, что пропущенных значений больше нет

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

среди строк с пропусками % задолженности такой же, как и в полной базе (8%).

- при ответе на первичный вопрос исследования эти строки не будут иметь большой вес (поля "дети" и "семейное положение" полностью заполнены): 
- их мало: 170 пустых NaN из 1741 строк с задолженностью и 101 с нулевым возрастом

поэтому пропуски можно заполнить:
- средний возраст вместо нулевых значений 'dob_years'
- возраст минус 18 лет - в поле со стажем работы 'days_employed'
- средний доход по типу занятости в столбце 'total_income'

нужно выяснить причину аномально больших данных в стаже - опечатка на порядок или это было расчетное значение в минутах?

### Замена типа данных

заменить вещественный тип данных на целочисленный в столбцах 'days_employed', 'dob_years'  и 'total_income'\
т.к. все пропуски заполнены, можно использовать простой метод astype('int')

In [ ]:
df['days_employed'] = df['days_employed'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info() # проверка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

тип данных изменен методом astype('int')

### Обработка дубликатов

In [ ]:
df.duplicated().sum() # 72 строки дубликата

72

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

**Вывод**

дубликаты удалены методом drop_duplicates() с переиндексацией строк\
после обработки пропусков дубликатов стало больше - с 54 до 72 строк\
возможная причина появления дубликатов - предыдущая очистка данных в базе

### Лемматизация

In [ ]:
df['purpose'].unique() # просмотр уникальных значений в столбце с целями получения кредита

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [ ]:
text_all = ' '.join(df['purpose']) # склейка всех значений столбца в строку

In [ ]:
#from pymystem3 import Mystem
#m = Mystem()

lemmas = m.lemmatize(text_all) # поиск лемм
print(lemmas) 

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'покупка', ' ', 'жилье', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'сыграть', ' ', 'свадьба', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'недвижимость', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'сыграть', ' ', 'свадьба', ' ', 'опера

In [ ]:
#from collections import Counter
print(Counter(lemmas)) # подсчет количества по каждой лемме

Counter({' ': 55021, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2323, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 764, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [ ]:
6367+4473+1881 # сумма упоминаний про недвижимость+жилье+строительство

12721

**Вывод**

на первом месте по запросам на предоставление кредита - недвижимость+жилье+строительство 12721 раз встречаются.

второе место разделили почти поровну - автомобиль 4315 и образование 4022.\

на третьем месте - свадьба 2348 раз.

- недвижимость 6367
- жилье 4473
- автомобиль 4315
- образование 4022
- свадьба 2348
- строительство 1881

### Категоризация данных

In [ ]:
def category(purpose):
    dictionery = ['недвижимость', 'жилье', 'строительство', 'автомобиль', 'образование', 'свадьба', 'другое']
    lemmasss = m.lemmatize(purpose)
    for word in dictionery:
        if word in lemmasss:
            return word

In [ ]:
df['purpose_category'] = df['purpose'].apply(category)

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,161380,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,161380,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,161380,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,161380,дополнительное образование,образование
4,0,3402,53,среднее,1,гражданский брак,1,F,пенсионер,0,139725,сыграть свадьбу,свадьба


In [ ]:
df['purpose_category'].value_counts()/21525*100

недвижимость    29.505226
жилье           20.720093
автомобиль      20.004646
образование     18.643438
свадьба         10.792102
Name: purpose_category, dtype: float64

**Вывод**

выделены 5 категорий:
- 29% недвижимость (может быть и коммерческой)
- 21% жилье (ремонт, покупка, строительство)
- 20% автомобиль
- 19% образование
- 11% свадьба

## Шаг 3. Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# посмотрим выборку имеющих задолженность в разрезе этих двух параметров
# должники есть во всех кластерах
data_pivot1 = df.pivot_table( index = 'family_status', columns = 'children', values = 'debt', aggfunc = 'sum')
data_pivot1

children,0,1,2,3,4,5
family_status,,,,,,
Не женат / не замужем,210.0,52.0,10.0,1.0,1.0,NaN
в разводе,55.0,21.0,8.0,1.0,0.0,NaN
вдовец / вдова,53.0,7.0,3.0,0.0,0.0,NaN
гражданский брак,229.0,118.0,33.0,8.0,0.0,0.0
женат / замужем,516.0,247.0,148.0,17.0,3.0,0.0


In [ ]:
#data_pivot1.transform(lambda x: round(x/x.sum()*100,2))

In [ ]:
# распределение должников по количеству детей - примерно одинаковый % во всех группах, кроме 4 детей - но здесь возможно погрешность из-за малого размера выборки
data_pivot11 = df.pivot_table( index = 'children', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot11['%_debt_vs_total'] = round(data_pivot11['sum', 'debt'] / data_pivot11['count', 'debt'],2)
data_pivot11

,count,sum,%_debt_vs_total
,debt,debt,
children,,,
0,14090,1063,0.08
1,4855,445,0.09
2,2128,202,0.09
3,330,27,0.08
4,41,4,0.10
5,9,0,0.00


**Вывод**

в целом наличие детей не влияет на риск возникновения задолженности по кредиту, но в совокупности с другими факторами может иметь значение. % должников по всем группам соответствует среднему по базе.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot2 = df.pivot_table( index = 'family_status', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot2['%_debt_vs_total'] = round(data_pivot2['sum', 'debt'] / data_pivot2['count', 'debt'],2)
data_pivot2
#в целом процент имеющих задолженность клиентов почти одинаковый по всем группам семейного положения, несколько выделяются 2 категории: 
#неженатые с высоким показателем = 10%, 
#у разведенных и вдовцов самый низкий показатель = 7%. 

,count,sum,%_debt_vs_total
,debt,debt,
family_status,,,
Не женат / не замужем,2810,274,0.10
в разводе,1195,85,0.07
вдовец / вдова,959,63,0.07
гражданский брак,4150,388,0.09
женат / замужем,12339,931,0.08


In [ ]:
#data_pivot2.transform(lambda x: round(x/x.sum()*100,2))

In [ ]:
data_pivot3 = df.pivot_table( index = ['family_status','children'], values = 'debt', aggfunc = ['count','sum'])
data_pivot3['%_debt_vs_total'] = round(100*(data_pivot3['sum', 'debt'] / data_pivot3['count', 'debt']),0)
data_pivot3
# относительно небольшая выборка, но стоит обратить внимание на следующие группы клиентов, по которым риск возникновения задолженности немного выше, чем у других заявителей:
# неженатые с детьми - 11-12%
# в разводе или вдовцы с 2-3 детьми - 10-12%
# гражданский брак с одним ребенком - 12%, при этом с 2 детьми клиенты намного стабильнее = 9%, что близко к среднему по всей базе. 
# Многодетные в гражданском или официальном браке (3-4 ребенка) также имеют высокий процент риска (10-14%), но таких данных немного - возможна стат погрешность.
#самый низкий процент должников - вдовцы без детей 6%, женатые или в разводе без детей 7%

count  sum %_debt_vs_total
                                debt debt                
family_status         children                           
Не женат / не замужем 0         2262  210             9.0
                      1          454   52            11.0
                      2           84   10            12.0
                      3            8    1            12.0
                      4            2    1            50.0
в разводе             0          784   55             7.0
                      1          316   21             7.0
                      2           83    8            10.0
                      3           11    1             9.0
                      4            1    0             0.0
вдовец / вдова        0          847   53             6.0
                      1           81    7             9.0
                      2           24    3            12.0
                      3            6    0             0.0
                      4            1    0             0.0
гражданский брак      0         2729  229             8.0
                      1         1000  118            12.0
                      2          355   33             9.0
                      3           56    8            14.0
                      4            8    0             0.0
                      5            2    0             0.0
женат / замужем       0         7468  516             7.0
                      1         3004  247             8.0
                      2         1582  148             9.0
                      3          249   17             7.0
                      4           29    3            10.0
                      5            7    0             0.0

**Вывод**

в целом процент имеющих задолженность клиентов почти одинаковый по всем группам семейного положения, несколько выделяются 2 категории: 
 - неженатые с высоким показателем = 10%, 
 - у разведенных и вдовцов самый низкий показатель = 7%. 

относительно небольшая выборка, но стоит обратить внимание на следующие группы клиентов, по которым риск возникновения задолженности немного выше, чем у других заявителей:
- неженатые с детьми - 11-12%
- в разводе или вдовцы с 2-3 детьми - 10-12%
- гражданский брак с одним ребенком - 12%, при этом с 2 детьми клиенты намного стабильнее = 9%, что близко к среднему по всей базе. 
- Многодетные в гражданском или официальном браке (3-4 ребенка) также имеют высокий процент риска (10-14%), но таких данных немного - возможна стат погрешность.
- самый низкий процент должников - вдовцы без детей 6%, женатые или в разводе без детей 7%

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
def inc_group(number):
    if number >= df['total_income'].mean(): return 'доход выше среднего'
    else: return 'доход ниже среднего'

In [ ]:
df['income_group'] = df['total_income'].apply(inc_group) # создаем новый столбец с категорией дохода выше\ниже среднего

In [ ]:
data_pivot4 = df.pivot_table( index = 'income_group', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot4['%_debt_vs_total'] = round(data_pivot4['sum', 'debt'] / data_pivot4['count', 'debt'],2)
data_pivot4

,count,sum,%_debt_vs_total
,debt,debt,
income_group,,,
доход выше среднего,6537,462,0.07
доход ниже среднего,14916,1279,0.09


так как уровень доходов сильно отличается в зависимости от типа занятости, проверим еще и этот показатель

In [ ]:
# так как уровень доходов сильно отличаетя в зависимости от типа занятости, проверим еще и этот показатель
data_pivot5 = df.pivot_table( index = 'income_type', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot5['%_debt_vs_total'] = round(data_pivot5['sum', 'debt'] / data_pivot5['count', 'debt'],2)
data_pivot5

,count,sum,%_debt_vs_total
,debt,debt,
income_type,,,
безработный,2,1,0.50
в декрете,1,1,1.00
госслужащий,1457,86,0.06
компаньон,5078,376,0.07
пенсионер,3828,216,0.06
предприниматель,2,0,0.00
сотрудник,11084,1061,0.10
студент,1,0,0.00


**Вывод**

среди пенсионеров и госслужащих мало должников - всего 6%, тогда как 10% сотрудников имеют задолженность.\
Клиенты с доходом ниже среднего уровня чаще имеют задолженность = 9%

### Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot6 = df.pivot_table( index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot6['%_debt_vs_total'] = round(data_pivot6['sum', 'debt'] / data_pivot6['count', 'debt'],2)
data_pivot6

,count,sum,%_debt_vs_total
,debt,debt,
purpose_category,,,
автомобиль,4306,403,0.09
жилье,4460,308,0.07
недвижимость,6351,474,0.07
образование,4013,370,0.09
свадьба,2323,186,0.08


**Вывод**

по сравнению со средними данными немного больше должников среди тех, кто брал кредит на покупку\ремонт автомобиля или на образование - 9% должников.\
вместе с тем, кредиты на жилье и недвижимость оплачиваются более регулярно - только 7% должников.

### Как образование влияет на возврат кредита в срок?

In [ ]:
data_pivot6 = df.pivot_table( index = 'education', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot6['%_debt_vs_total'] = round(data_pivot6['sum', 'debt'] / data_pivot6['count', 'debt'],2)
data_pivot6

,count,sum,%_debt_vs_total
,debt,debt,
education,,,
высшее,5249,278,0.05
начальное,282,31,0.11
неоконченное высшее,744,68,0.09
среднее,15172,1364,0.09
ученая степень,6,0,0.00


**Вывод**

Клиенты с высшим образованием реже имеют задолженность - всего 5%, тогда как с начальным образованием = 11%

## Шаг 4. Общий вывод

**Характеристики клиентов банка:**
- женщин в 2 раза больше, чем мужчин (14236 / 7288),
- 70% со средним образованием (свыше 15000 строк), 
- 77% cостоят в браке (12380+4177 гражданский брак),
- 66% клиентов без детей (14149 из 21525 строк), 
- 58% работают по найму (11119+1459 госслужащих), 24% =компаньоны (5085), достаточно много пенсионеров = 18% (3856),
- возраст заемщика от 19 до 75 лет, средний = 43, начало трудовой деятельности после 27 лет,
- ежемесячный доход от 20тр до 2 млн рублей, среднее = 167тр, и только 2% (528 строк/клиентов) с доходом >500тр,
задолженность есть только у 8% клиентов (1741 от 21524).

**Цели кредита можно объединить в 5 категорий:**
- 29% недвижимость (может быть и коммерческой)
- 21% жилье (ремонт, покупка, строительство)
- 20% автомобиль
- 19% образование
- 11% свадьба

**Как разные параметры влияют на возврат кредита в срок?**

Наиболее сильно влияют образование, тип занятости и уровень дохода:
- Клиенты с высшим образованием реже имеют задолженность - всего 5%, тогда как с начальным образованием = 11%
- среди пенсионеров и госслужащих мало должников - всего 6%, тогда как 10% сотрудников имеют задолженность.
- клиенты с доходом ниже среднего уровня чаще имеют задолженность = 9%

Также важна совокупность семейного положения и количества детей у заемщика - стоит обратить внимание на следующие группы клиентов, по которым риск возникновения задолженности немного выше, чем у других заявителей:
- неженатые с детьми - 11-12%
- в разводе или вдовцы с 2-3 детьми - 10-12%
- гражданский брак с одним ребенком - 12%, при этом с 2 детьми клиенты намного стабильнее = 9%, что близко к среднему по всей базе. 
- Многодетные в гражданском или официальном браке (3-4 ребенка) также имеют высокий процент риска (10-14%), но таких данных немного - возможна стат погрешность.
- самый низкий процент должников - вдовцы без детей 6%, женатые или в разводе без детей 7%

Цель кредиты меньше влияет: немного больше должников среди тех, кто брал кредит на покупку\ремонт автомобиля или на образование - 9% должников. Вместе с тем, кредиты на жилье и недвижимость оплачиваются более регулярно - только 7% должников.